## SYMPTOMS_Identifier

In [1]:
import spacy

In [2]:
from tqdm import tqdm
import spacy
import random
import json
import re

In [3]:
training_data = []
lines=[]
with open('Copy of Trained_Advice.json', 'r',encoding="utf8") as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
        for annotation in data['annotation']:
              if(annotation is not None):
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [4]:
len(training_data)

18

In [5]:
training_data[4:10]

[('Abhijit kaushal, your age is 29 yeras.I have the symptoms of High fever and Nousia.The problem i found that you are  suffering from Tetanus.The prescribed medicine you have to take are pyrantel,Tablet (chewable): 250 mg (as embonate orpamoate).You should try Physical activity which promotes blood circulation.',
  {'entities': [(75, 82, 'ADVICES'), (61, 71, 'ADVICES')]}),
 ('Hi Mr Avnish Chaurasia of age 23 years.I have the problems of Unexpected weight loss,night sweats.You are having the Congenital Heart Disease.According to report What i found is that you are suffering from Tuberculosis.The prescribed medicine you have to take are chloramphenicol Oily suspension for injection*: 0.5 g (as sodiumsuccinate)/ mL in 2‐ mL ampoule,erythromycin Powder for oral liquid: 125 mg/5 mL (as stearate or estolate or ethyl succinate).You should flex your ankles every half hour or so while sitting.You should try Avoiding foods that trigger indigestion.Drink Warm water.',
  {'entities': [(85, 97, 'A

In [6]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 18/18 [00:00<00:00, 944.91it/s]


Statring iteration 0
{'ner': 433.6112278322084}
Statring iteration 1
{'ner': 83.17658944897795}
Statring iteration 2
{'ner': 627.9742011487976}
Statring iteration 3
{'ner': 104.63287142088308}
Statring iteration 4
{'ner': 70.95102622901634}
Statring iteration 5
{'ner': 38.044413910092814}
Statring iteration 6
{'ner': 15.508880126525284}
Statring iteration 7
{'ner': 10.681692982390285}
Statring iteration 8
{'ner': 5.7621844684923165}
Statring iteration 9
{'ner': 10.015610119999167}
Statring iteration 10
{'ner': 2.1369369694850553}
Statring iteration 11
{'ner': 4.865382472040152}
Statring iteration 12
{'ner': 0.0804825457125184}
Statring iteration 13
{'ner': 0.0626104467821548}
Statring iteration 14
{'ner': 0.13805090441955828}
Statring iteration 15
{'ner': 0.0005517704711312697}
Statring iteration 16
{'ner': 6.7375369556608845e-06}
Statring iteration 17
{'ner': 4.95542507885632e-05}
Statring iteration 18
{'ner': 0.042165572630572544}
Statring iteration 19
{'ner': 0.0003521413777108464}


In [7]:
import spacy

In [8]:
nlp.to_disk("new_model_3")

In [14]:
import spacy
nlp4=spacy.load("new_model_3")

In [15]:
doc=nlp4(open("./MEDICAL_REPORTS/TEST_REPORT_11.txt").read())

In [16]:
print(doc)

hello,Piyush Jagga, You are 25 years old.i am suffering from problems of Persistent lumps,swollen glands.I am suffering from problem of disease HIV.Medicines diazepam Gel or rectal solution: 5 mg/ mL in 0.5 mL 2‐ mL 4‐mL tubes,lorazepam Parenteral formulation: 2 mg/ mL in 1‐ mL ampoule; 4 mg/ mL in 1‐ mL ampoule.avoid lifting heavy weights.You should maintain your Body Mass Index.I would suggest doing more of effort.


In [17]:
for entity in doc.ents:
    print(entity.text)

problems of Persistent lumps
swollen glands
